In [1]:
# HMM_LDA
# ライブラリをインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import gensim
import itertools
from numpy.random import *
from scipy import optimize
from scipy import sparse
import seaborn as sns

In [2]:
# 多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

# データの生成

## 入力データの定義

In [3]:
# データの設定
k1 = 8   # syntax数
k2 = 15   # トピック数
d = 3000   # 文書数
v1 = 800   # トピックに関係のある語彙数
v2 = 500   # syntaxに関係のある語彙数
v = v1 + v2   # 総語彙数
w = np.random.poisson(np.random.gamma(42.5, 1/0.15, d))   # 文書あたりの単語数
f = np.sum(w)   # 総単語数
vec_k1 = np.repeat(1, k1)
vec_k2 = np.repeat(1, k2)

In [4]:
# IDとインデックスの設定
# IDの設定
d_id = np.repeat(np.arange(d), w)
pt_id = np.array(list(itertools.chain(*[np.array(range(w[i]), dtype="int") for i in range(d)])))

#インデックスの設定
index = np.arange(f)
d_list = [i for i in range(d)]
d_vec = [i for i in range(d)]
for i in range(d):
    d_list[i] = index[d_id==i]
    d_vec[i] = np.repeat(1, w[i])

In [5]:
# パラメータの事前分布の設定
# ディリクレ分布の事前分布
alpha01 = np.append(np.arange(2.5, 0.5, -(2.5-0.5)/(k1-1)), 0.5)
alpha02 = np.full((k1, k1), 0.5)
alpha02[np.delete(np.arange(k1), k1-1), k1-1] = 7.5; alpha02[k1-1, k1-1] = 3.0
alpha11 = np.repeat(0.15, k2)
alpha21 = np.append(np.repeat(0.05, v1), np.repeat(0.00025, v2))

# syntaxの事前分布
alloc = np.dot(np.random.multinomial(1, np.random.dirichlet(np.repeat(5.0, k1-1), 1)[-1], v2), np.arange(k1-1))
alpha22 = np.hstack((np.full((k1-1, v1), 0.0001), np.full((k1-1, v2), 0.0025)))
for j in range(k1-1):
    index = np.where(alloc==j)[0] + v1
    alpha22[j, index] = 2.5

## 応答変数の生成

In [6]:
# モデルに基づき単語を生成
rp = 0
while True:
    rp = rp + 1
    print(rp)
    
    # ディリクレ分布よりパラメータを生成
    pi1 = np.random.dirichlet(alpha01, 1).reshape(-1)
    pi2 = np.zeros((k1, k1))
    for j in range(k1):
        pi2[j, ] = np.random.dirichlet(alpha02[j, ], 1)
    theta = np.random.dirichlet(alpha11, d)
    phi = np.random.dirichlet(alpha21, k2)
    psi = np.zeros((k1-1, v))
    for j in range(k1-1):
        psi[j, ] = np.random.dirichlet(alpha22[j, ], 1)
    pit1 = pi1.copy(); pit2 = pi2.copy(); thetat = theta.copy()

    # 単語出現確率が低いトピックを入れ替える
    index1 = np.where(np.max(psi, axis=0) < (k1*10)/f)[0]; index1 = index1[index1 >= v1] 
    index2 = np.where(np.max(phi, axis=0) < (k2*10)/f)[0]; index2 = index2[index2 < v1]
    for j in range(index1.shape[0]):
        psi[np.argmax(np.random.multinomial(1, np.repeat(1/(k1-1), k1-1), 1)), index1[j]] = (k1*10)/f
    psi = psi / np.sum(psi, axis=1)[:, np.newaxis]
    for j in range(index2.shape[0]):
        phi[np.argmax(np.random.multinomial(1, np.repeat(1/k2, k2), 1)), index2[j]] = (k2*10)/f
    phi = phi / np.sum(phi, axis=1)[:, np.newaxis]
    psit = psi.copy(); phit = phi.copy()

    # HMM-LDAに基づき単語を生成
    WX = np.array(np.zeros((d, v)), dtype="int")
    wd_list = [i for i in range(d)]
    Z1_list = [i for i in range(d)]
    Z2_list = [i for i in range(d)]

    for i in range(d):
        z1 = np.array(np.zeros((w[i], k1)), dtype="int")
        z2 = np.array(np.zeros((w[i], k2)), dtype="int")
        z2_vec = np.repeat(0, w[i])
        words = np.array(np.zeros((w[i], v)), dtype="int")

        for j in range(w[i]):
            if j==0:
                # 文書の先頭の単語のsyntaxを生成
                z1[j, ] = np.random.multinomial(1, pi1, 1)
            else:
                # 先頭以降はマルコフ推移に基づきsyntaxを生成
                z1[j, ] = np.random.multinomial(1, pi2[np.argmax(z1[j-1, ]), ], 1)
        z1_vec = np.dot(z1, np.arange(k1))

        # トピック分布を生成
        index_topic = np.where(z1_vec==k1-1)[0]
        z2[index_topic, ] = np.random.multinomial(1, theta[i, ], index_topic.shape[0])
        z2_vec[index_topic] = np.dot(z2[index_topic, ], np.arange(k2)) + 1

        # 多項分布から単語を生成
        index_syntax = np.delete(np.arange(w[i]), index_topic)
        for j in range(index_syntax.shape[0]):
            words[index_syntax[j], ] = np.random.multinomial(1, psi[z1_vec[index_syntax[j]], ], 1)
        for j in range(index_topic.shape[0]):
            words[index_topic[j], ] = np.random.multinomial(1, phi[(z2_vec-1)[index_topic[j]], ], 1)

        #データを格納
        wd_list[i] = np.dot(words, np.arange(v))
        WX[i, ] = np.sum(words, axis=0)
        Z1_list[i] = z1
        Z2_list[i] = z2

    if np.min(np.sum(WX, axis=0)) > 0:
        break

1


In [7]:
# 生成したデータを変換
# リストを変換
wd = np.array(list(itertools.chain(*[wd_list[i] for i in range(d)])))
Z1 = np.array(list(itertools.chain(*[Z1_list[i] for i in range(d)])))
Z2 = np.array(list(itertools.chain(*[Z2_list[i] for i in range(d)])))
del wd_list; del Z1_list; del Z2_list

In [8]:
# スパース行列を定義
sparse_data = sparse.coo_matrix((np.repeat(1, f), (np.arange(f), wd)), shape=(f, v)).tocsr()
sparse_data_T = sparse_data.T
d_dt = sparse.coo_matrix((np.repeat(1, f), (d_id, np.arange(f))), shape=(d, f)).tocsr()

In [9]:
# 単語インデックスを作成
w_list = [j for j in range(v)]
w_vec = [j for j in range(v)]
for j in range(v):
    w_list[j] = np.array(np.where(wd==j)[0], dtype="int")
    w_vec[j] = np.repeat(1, w_list[j].shape[0])

# HMM-LDAを推定

## アルゴリズムを定義

In [10]:
# マルコフ連鎖モンテカルロ法でHMM-LDAを推定
# トピック尤度と負担率を計算する関数
def LLho(theta, phi, d_id, wd, f, k):
    Lho = theta[d_id, ] * (phi.T)[wd, ]
    topic_rate = Lho / np.sum(Lho, axis=1).reshape(f, 1)
    return Lho, topic_rate

In [11]:
# インデックスの設定
# 先頭と後尾のインデックスを作成
max_word = np.max(pt_id) + 1
index_t11 = np.array(np.where(pt_id==0)[0], dtype="int")
index_t12 = np.repeat(0, d)
for i in range(d):
    index_t12[i] = np.max(d_list[i])

In [12]:
# 中間のインデックスを作成
d_id0 = np.append(d_id, d)
index_list_t21 = [j for j in range(max_word-1)]
index_list_t22 = [j for j in range(max_word-1)]
for j in range(1, max_word):
    index_list_t21[j-1] = np.array(np.where(pt_id==j)[0], dtype="int") - 1
    index_list_t22[j-1] = np.array(np.where(pt_id==j)[0], dtype="int")
index_t21 = np.sort(np.array(list(itertools.chain(*[index_list_t21[j] for j in range(max_word-1)]))))
index_t22 = np.sort(np.array(list(itertools.chain(*[index_list_t22[j] for j in range(max_word-1)]))))

In [13]:
# アルゴリズムの設定
R = 1000   # サンプリング回数
keep = 2   # 2回に1回の割合でサンプリング結果を格納
disp = 50
iter = 0
burnin = int(500/keep)

# 事前分布の設定
alpha01 = 0.2
alpha02 = 0.1
alpha03 = 0.1
beta01 = 0.1
beta02 = 0.1

In [14]:
# パラメータの真値
# トピック分布とマルコフ推移行列の真値
theta = thetat.copy()
pi1 = pit1.copy()
pi2 = pit2.copy()

# 単語分布の真値
psi = psit.copy()
phi = phit.copy()

# HMMの潜在変数の真値
Zi1 = Z1.copy()
z1 = np.dot(Zi1, np.arange(k1))

In [15]:
# パラメータの初期値
# トピック分布とマルコフ推移行列の初期値
theta = np.random.dirichlet(np.repeat(100.0, k2), d)
pi1 = np.random.dirichlet(np.repeat(100.0, k1), 1)
pi2 = np.random.dirichlet(np.append(np.repeat(30, k1-1), 100.0), k1)

# 単語分布の初期値
psi = np.random.dirichlet(np.repeat(100.0, v), k1-1)
phi = np.random.dirichlet(np.repeat(100.0, v), k2)

# HMMの潜在変数の初期値
Zi1 = np.random.multinomial(1, np.repeat(1/k1, k1), f)
z1 = np.dot(Z1, np.arange(k1))

In [16]:
# パラメータの格納用配列
# モデルパラメータの格納用配列
THETA = np.zeros((d, k2, int(R/keep)))
PI1 = np.zeros((int(R/keep), k1))
PI2 = np.zeros((k1, k1, int(R/keep)))
PSI = np.zeros((k1-1, v, int(R/keep)))
PHI = np.zeros((k2, v, int(R/keep)))

# トピックの格納用配列
SEG1 = np.zeros((f, k1))
SEG2 = np.zeros((f, k2))

In [17]:
# 対数尤度の基準値
# ユニグラムモデルの対数尤度
par = np.sum(WX, axis=0) / f
LLst = np.sum(np.dot(WX, np.log(par)))

# 真値での対数尤度
z1 = np.dot(Z1, np.arange(k1))
index_topic = np.array(np.where(z1==k1-1)[0], dtype="int")
index_syntax = np.delete(np.arange(f), index_topic)
LLbest1 = np.sum(np.log(np.sum(Z1[index_syntax, :k1-1] * (psit.T)[wd[index_syntax], ], axis=1)))
LLbest2 = np.sum(np.log(np.sum((thetat[d_id, ] * (phit.T)[wd, ])[index_topic, ], axis=1)))
LLbest = LLbest1 + LLbest2
print(np.array([LLst, LLbest]))

[-5677335.84063493 -4053001.76190603]


## パラメータを推定

In [18]:
# ギブスサンプリングでパラメータをサンプリング
for rp in range(R):
    
    # 単語ごとの尤度と混合率を設定
    # syntaxとトピックモデルの尤度
    Lho1 = (psi.T)[wd, ]   # syntaxごとの尤度
    Lho2 = theta[d_id, ] * (phi.T)[wd, ]
    Lho_mu2 = np.dot(Lho2, vec_k2)   # トピックモデルの期待尤度
    Lho = np.hstack((Lho1, Lho_mu2[:, np.newaxis]))   # 尤度の結合

    # HMMの混合率と推移確率
    pi_dt1 = np.full((f, k1), 1/k1); pi_dt2 = pi_dt1.copy()
    pi_dt1[index_t11, ] = np.full((d, k1), pi1)   # 文書の先頭の混合率
    pi_dt1[index_t22, ] = pi2[z1[index_t21], ]   # 1単語前の混合率
    pi_dt2[index_t21, ]= (pi2.T)[z1[index_t22], ]   # 1単語後の混合率

    # 多項分布からHMMの潜在変数をサンプリング
    # 潜在変数の割当確率
    Posterior = pi_dt1 * pi_dt2 * Lho   # 結合分布
    Prob = Posterior / np.dot(Posterior, vec_k1)[:, np.newaxis]

    # 潜在変数をサンププリング
    Zi1 = rmnom(Prob, f, k1, 1)[1]
    z1 = np.dot(Zi1, np.arange(k1))
    n1 = np.sum(Zi1[:, :k1-1])
    n2 = np.sum(Zi1[:, k1-1])
    index_syntax = np.where(z1!=k1-1)[0].astype("int")
    index_topic = np.where(z1==k1-1)[0].astype("int")


    # HMMのパラメータをサンプリング
    # ディリクレ分布から推移確率のパラメータをサンプリング
    rf11 = np.sum(Zi1[index_t11, ], axis=0) + alpha01
    rf12 = np.dot(Zi1[index_t21, ].T, Zi1[index_t22, ]) + alpha02
    pi1 = np.random.dirichlet(rf11, 1).reshape(-1)
    pi2 = np.zeros((k1, k1))
    for j in range(k1):
        pi2[j, ] = np.random.dirichlet(rf12[j, ], 1).reshape(-1)

    # トピックモデルのパラメータをサンプリング
    # トピックをサンプリング
    Prob = Lho2[index_topic, ] / np.dot(Lho2[index_topic, ], vec_k2)[:, np.newaxis]
    Zi2 = np.array(np.zeros((f, k2)), dtype="int")
    Zi2[index_topic, ] = rmnom(Prob, n2, k2, 1)[1]
    
    # トピック分布のパラメータをサンプリング
    theta = np.zeros((d, k2))
    for i in range(d):
        wsum = np.dot(Zi2[d_list[i]].T, d_vec[i]) + alpha02
        theta[i, ] = np.random.dirichlet(wsum, 1).reshape(-1)

        
    # 単語分布をサンプリング
    # ディリクレ分布から単語分布のパラメータをサンプリング
    vf1 = np.zeros((k1-1, v)); vf2 = np.zeros((k2, v))
    psi = np.zeros((k1-1, v)); phi = np.zeros((k2, v))
    for j in range(v):
        vf1[:, j] = np.dot(Zi1[w_list[j], :k1-1].T, w_vec[j]) + beta01
        vf2[:, j] = np.dot(Zi2[w_list[j], ].T, w_vec[j]) + beta02
    for j in range(k1-1):
        psi[j, ] = np.random.dirichlet(vf1[j, ], 1).reshape(-1)
    for j in range(k2):
        phi[j, ] = np.random.dirichlet(vf2[j, ], 1).reshape(-1)


    # パラメータの格納とサンプリング結果の表示
    # サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        THETA[:, :, mkeep] = theta
        PI1[mkeep, ] = pi1
        PI2[:, :, mkeep] = pi2
        PHI[:, :, mkeep] = phi
        PSI[:, :, mkeep] = psi

    # トピック割当はバーンイン期間を超えたら格納
    if rp%keep==0 & rp >= burnin:
        SEG1 = SEG1 + Zi1
        SEG2 = SEG2 + Zi2

    if rp%disp==0:
        # 対数尤度の更新
        LL1 = np.sum(np.log(np.sum(Zi1[index_syntax, :k1-1] * (psi.T)[wd[index_syntax], ], axis=1)))
        LL2 = np.sum(np.log(np.sum((theta[d_id, ] * (phi.T)[wd, ])[index_topic, ], axis=1)))
        LL = LL1 + LL2

        #サンプリング結果を確認
        print(rp)
        print(np.round(np.array([LL, LLst, LLbest]), 1))

0
[-5674398.7 -5677335.8 -4053001.8]
50
[-4609443.4 -5677335.8 -4053001.8]
100
[-4206784.4 -5677335.8 -4053001.8]
150
[-4162548.9 -5677335.8 -4053001.8]
200
[-4154573.  -5677335.8 -4053001.8]
250
[-4144238.4 -5677335.8 -4053001.8]
300
[-4142069.9 -5677335.8 -4053001.8]
350
[-4119758.4 -5677335.8 -4053001.8]
400
[-4119839.2 -5677335.8 -4053001.8]
450
[-4120482.7 -5677335.8 -4053001.8]
500
[-4114161.1 -5677335.8 -4053001.8]
550
[-4112156.  -5677335.8 -4053001.8]
600
[-4114614.5 -5677335.8 -4053001.8]
650
[-4113964.3 -5677335.8 -4053001.8]
700
[-4115507.1 -5677335.8 -4053001.8]
750
[-4112388.6 -5677335.8 -4053001.8]
800
[-4112489.8 -5677335.8 -4053001.8]
850
[-4110735.3 -5677335.8 -4053001.8]
900
[-4108119.8 -5677335.8 -4053001.8]
950
[-4112474.3 -5677335.8 -4053001.8]
